In [1]:
#imports
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

Using Theano backend.
WARNING (theano.gof.compilelock): Overriding existing lock by dead process '6636' (I am process '11080')
DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/Joris/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmp6oxgbq/265abc51f7c376c224983485238ff1a5.lib and object C:/Users/Joris/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmp6oxgbq/265abc51f7c376c224983485238ff1a5.exp

Using gpu device 0: GeForce GTX 1060 6GB (CNMeM is disabled, cuDNN 5105)
D:\Anaconda\envs\python2\lib\site-packages\theano\sandbox\cuda\__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
# Paths
TRAIN_FOLDER = 'E:/Data/train'
TEST_FOLDER = 'E:/Data/test_stg1'
AUGMENT_FOLDER = None # folder to save augmented image for testing

# network config
input_size = 256
filter1_size = 32
filter2_size = 64 
classes = 8
batch_size = 32
epochs=10

In [3]:
def keras_model_conv():
    model = Sequential()
    model.add(Convolution2D(filter1_size, 3, 3, 
                          border_mode='same', 
                          input_shape=(3, input_size, input_size)))
    model.add(Activation('relu'))
    model.add(Convolution2D(filter1_size, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Convolution2D(filter2_size, 3, 3))
    model.add(Activation('relu'))
    model.add(Convolution2D(filter2_size, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy', 'categorical_crossentropy'])

    return model


In [4]:
datagen = ImageDataGenerator(horizontal_flip=True,
                            rescale=1./255.,
                            rotation_range=10)
gen = datagen.flow_from_directory(TRAIN_FOLDER,
                            target_size=(input_size,input_size),
                            batch_size=batch_size,
                            save_to_dir=AUGMENT_FOLDER)



Found 3777 images belonging to 8 classes.


In [5]:
model = keras_model_conv()
history = model.fit_generator(gen,
                              samples_per_epoch=1000,
                              nb_epoch=epochs,
                              verbose=1 
                              )


Epoch 1/10
1024/1000 [==============================] - 56s - loss: 8.7371 - acc: 0.4375 - categorical_crossentropy: 8.7371    

D:\Anaconda\envs\python2\lib\site-packages\keras\engine\training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '



Epoch 2/10
1024/1000 [==============================] - 53s - loss: 8.8303 - acc: 0.4521 - categorical_crossentropy: 8.8303    
Epoch 3/10
1024/1000 [==============================] - 53s - loss: 8.6729 - acc: 0.4619 - categorical_crossentropy: 8.6729    
Epoch 4/10
1025/1000 [==============================] - 50s - loss: 8.8689 - acc: 0.4498 - categorical_crossentropy: 8.8689    
Epoch 5/10
1024/1000 [==============================] - 44s - loss: 8.6729 - acc: 0.4619 - categorical_crossentropy: 8.6729    
Epoch 6/10
1024/1000 [==============================] - 44s - loss: 8.9877 - acc: 0.4424 - categorical_crossentropy: 8.9877    
Epoch 7/10
1024/1000 [==============================] - 44s - loss: 8.3581 - acc: 0.4814 - categorical_crossentropy: 8.3581    
Epoch 8/10
1025/1000 [==============================] - 44s - loss: 9.0104 - acc: 0.4410 - categorical_crossentropy: 9.0104    
Epoch 9/10
1024/1000 [==============================] - 43s - loss: 8.7359 - acc: 0.4580 - categorical_

In [6]:
model.save('E:/models/simple.h5')